<a href="https://colab.research.google.com/github/somwrks/AI-Solar-Panel/blob/main/main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Self Rotatory Solar Panel

This Project utilizes various vision models to train on planet sun labelled datasets and perform validation checks to determine which model performs the best and converts the best model to tinyML Model to make it usable by microcontroller

## Installing Dependencies

In [ ]:
%pip install --upgrade pip


In [ ]:
%pip install inference tensorflow[and-cuda] tensorflow-datasets roboflow ultralytics torch torchvision matplotlib tqdm


## Importing Libraries

In [ ]:
import tensorflow as tf
import tensorflow_datasets as tfds
import torch
import torchvision  
from torchvision import transforms
from torch.utils.data import DataLoader, ConcatDataset
from ultralytics import YOLO
import matplotlib.pyplot as plt
from tqdm import tqdm
import shutil
from tensorflow.keras import mixed_precision
from roboflow import Roboflow
import yaml
import os
from PIL import Image


In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
os.environ['TF_ENABLE_ONEDNN_OPTS'] = '0' 
os.environ['NO_ALBUMENTATIONS_UPDATE '] = '1'  


In [ ]:
print("TensorFlow version:", tf.__version__)
print("GPU Available:", tf.config.list_physical_devices())


In [ ]:
# List available GPUs
physical_devices = tf.config.list_physical_devices('GPU')
print("Num GPUs Available: ", len(physical_devices))

if len(physical_devices) > 0:
# Enable memory growth for all GPUs
  for gpu in physical_devices:
      tf.config.experimental.set_memory_growth(gpu, True)

  # Use all available GPUs
  tf.config.set_visible_devices(physical_devices, 'GPU')

  # Enable XLA (Accelerated Linear Algebra) optimization
  tf.config.optimizer.set_jit(True)

  tf.config.set_visible_devices(physical_devices[0], 'GPU')

  # Enable mixed precision training

  policy = mixed_precision.Policy('mixed_float16')
  mixed_precision.set_global_policy(policy)

  print("GPU optimization settings applied successfully")
else:
    print("No GPU available. Running on CPU.")

## Loading Datasets

In [ ]:
import os
import yaml
from roboflow import Roboflow
from PIL import Image

# Download both datasets
rf = Roboflow(api_key="")

# Dataset 1

project1 = rf.workspace("yassine-pzpt7").project("sun-tracking-photovoltaic-panel")
version1 = project1.version(2)
dataset1 = version1.download("yolov8")

# Dataset 2
project2 = rf.workspace("1009727588-qq-com").project("sun-nxvfz")
dataset2 = project2.version(2).download("yolov8")

# Dataset 3
project3 = rf.workspace("rik-tjduw").project("sun-tracking-555mn")
version3 = project3.version(4)
dataset3 = version3.download("yolov8")

# Dataset 4
project4 = rf.workspace("stardetect").project("solar-re1fe")
version4 = project4.version(1)
dataset4 = version4.download("yolov8")

# Dataset 5
project5 = rf.workspace("fruitdetection-ulcz9").project("sun_detection-hl04q")
version5 = project5.version(1)
dataset5 = version5.download("yolov8")

# Prepare dataset paths
dataset1_path = dataset1.location
dataset2_path = dataset2.location
dataset3_path = dataset3.location
dataset4_path = dataset4.location
dataset5_path = dataset5.location

def resize_images(directory, target_resolution=(640, 640)):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    for root, _, files in os.walk(directory):
        for file in files:
            if file.lower().endswith(image_extensions):
                image_path = os.path.join(root, file)
                try:
                    with Image.open(image_path) as img:
                        img_resized = img.resize(target_resolution, Image.LANCZOS)
                        img_resized.save(image_path)
                except Exception as e:
                    print(f"Error resizing image {image_path}: {e}")

# Function to count images in a directory
def count_images(directory):
    image_extensions = ('.jpg', '.jpeg', '.png', '.bmp', '.tif', '.tiff')
    return len([f for f in os.listdir(directory) if f.lower().endswith(image_extensions)])

# Function to combine datasets

def combine_datasets(*dataset_paths, combined_path):
    for split in ['train', 'valid', 'test']:
        os.makedirs(os.path.join(combined_path, split, 'images'), exist_ok=True)
        os.makedirs(os.path.join(combined_path, split, 'labels'), exist_ok=True)

        for dataset_path in dataset_paths:
            images_src = os.path.join(dataset_path, split, 'images')
            labels_src = os.path.join(dataset_path, split, 'labels')

            images_dst = os.path.join(combined_path, split, 'images')
            labels_dst = os.path.join(combined_path, split, 'labels')

            if os.path.exists(images_src):
                for file in os.listdir(images_src):
                    shutil.copy2(os.path.join(images_src, file), images_dst)
            if os.path.exists(labels_src):
                for file in os.listdir(labels_src):
                    shutil.copy2(os.path.join(labels_src, file), labels_dst)


# Combine datasets
combined_path = './combined_dataset'
combine_datasets(dataset1_path, dataset2_path, dataset3_path,dataset4_path, dataset5_path, combined_path=combined_path)

# Resize images in the combined dataset
for split in ['train', 'valid', 'test']:
    images_dir = os.path.join(combined_path, split, 'images')
    resize_images(images_dir)

print("Images resized to target resolution.")


# Count images in each dataset
train_count = len(os.listdir(os.path.join(combined_path, 'train', 'images')))
valid_count = len(os.listdir(os.path.join(combined_path, 'valid', 'images')))
test_count = len(os.listdir(os.path.join(combined_path, 'test', 'images')))
num_classes = len(set([f.split('_')[0] for f in os.listdir(os.path.join(combined_path, 'train', 'labels'))]))

print(f"Number of images in training set: {train_count}")
print(f"Number of images in validation set: {valid_count}")
print(f"Number of images in test set: {test_count}")
print(f"Total number of images: {train_count + valid_count + test_count}")

# Create YAML file for YOLOv5
yaml_content = {
    'train': os.path.join(combined_path, 'train'),
    'val': os.path.join(combined_path, 'valid'),
    'test': os.path.join(combined_path, 'test'),
    'nc': len(os.listdir(os.path.join(combined_path, 'train', 'labels'))),
    'names': [f'class_{i}' for i in range(len(os.listdir(os.path.join(combined_path, 'train', 'labels'))))],
    'image_counts': {
        'train': train_count,
        'val': valid_count,
        'test': test_count
    }
}


yaml_content = {
    'train': os.path.join(combined_path, 'train'),
    'val': os.path.join(combined_path, 'valid'),
    'test': os.path.join(combined_path, 'test'),
    'nc': num_classes,
    'names': [f'class_{i}' for i in range(num_classes)],
}

yaml_path = os.path.join(combined_path, 'dataset.yaml')
with open(yaml_path, 'w') as yaml_file:
    yaml.dump(yaml_content, yaml_file, default_flow_style=False)

print(f"Dataset YAML file created at: {yaml_path}")


## Setting Up Models


### 1. YoloV5 (LibRT)

In [ ]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"


In [ ]:
from ultralytics import YOLO

yolo_model = YOLO('runs/detect/train6/weights/best.pt')

full_yaml_path = os.path.abspath(yaml_path)
yolo_results = yolo_model.train(
    data=full_yaml_path,
    epochs=1000,  
    imgsz=640,
    lr0=0.001,  
    lrf=0.0001, 
    batch=-1,  
    device=[0],
    augment=True,
    cos_lr=True,
    amp=True,  
    resume=True,
    patience=300,
    mixup=0.2,  
    mosaic=1.0, 
    close_mosaic=10  
)

# Evaluate the model after training
metrics = yolo_model.val()
print(f"mAP50-95: {metrics.box.map}")


## Testing Models

In [ ]:
# Evaluate YOLOv5
yolo_metrics = yolo_model.val()

print("YOLOv5 mAP:", yolo_metrics.results_dict['metrics/mAP50-95(B)'])


## Saving Model

In [ ]:
model.save('sun_tracker_model.h5')


## Pretrained Model Implementation

Static Models from roboflow web


| Model | mAP50-95 | Precision | Recall | Images | Type |
|-------|----------|-----------|--------|--------|------|
| Our YOLO model | 73.5% | 89.2% | 91.7% | 633 | YOLOv8 (custom trained) |
| solar-re1fe/1 | 96.8% | 95.2% | 94.3% | 2684 | Roboflow 3.0 Object Detection (Fast) |
| sun-tracking-photovoltaic-panel/1 | 98.2% | 93.7% | 93.7% | 196 | Roboflow 2.0 Object Detection (Fast) |
| sun-tracking/3 | 92.5% | 94.7% | 91.8% | 1090 | YOLOv5 Model Upload |
| sun-tracking-555mn/4 | 97.7% | 93.0% | 89.2% | 923 | YOLOv8n Model Upload |



1. **sun-tracking-555mn/4**
   - Best overall performance with 97.7% mAP
   - YOLOv8n architecture, which is lightweight and suitable for microcontrollers
   - Recently trained (2024-03-05)
   - Balanced dataset size (923 images)

2. **solar-re1fe/1**
   - High performance (96.8% mAP, 95.2% precision, 94.3% recall)
   - Largest dataset (2684 images), potentially good for generalization
   - Roboflow 3.0 Object Detection (Fast) may be suitable for microcontrollers

3. **sun-tracking-photovoltaic-panel/1**
   - Highest mAP (98.2%)
   - Roboflow 2.0 Object Detection (Fast) may work on microcontrollers
   - Limited dataset (196 images) might affect generalization

4. **sun-tracking/3**
   - Lowest mAP (92.5%) among the options
   - YOLOv5 architecture, which is generally suitable for microcontrollers
   - Decent dataset size (1090 images)


In [ ]:
from inference import get_model
import supervision as sv
import cv2

# define the image url to use for inference
image_file = "test.jpg"
image = cv2.imread(image_file)

# load a pre-trained yolov8n model
model = get_model(model_id="sun-tracking-555mn/4", api_key="")

# run inference on our chosen image, image can be a url, a numpy array, a PIL image, etc.
results = model.infer(image)[0]

# load the results into the supervision Detections api
detections = sv.Detections.from_inference(results)

# create supervision annotators
bounding_box_annotator = sv.BoxAnnotator()
label_annotator = sv.LabelAnnotator()

# annotate the image with our inference results
annotated_image = bounding_box_annotator.annotate(
    scene=image, detections=detections)
annotated_image = label_annotator.annotate(
    scene=annotated_image, detections=detections)

# display the image
sv.plot_image(annotated_image)